In [ ]:
!git clone https://github.com/LynnHaDo/Checkbox-Detection.git

fatal: destination path 'Checkbox-Detection' already exists and is not an empty directory.


In [ ]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install albumentations
!pip install ultralytics
!pip install gradio
!pip install torch
!pip install ruamel

ERROR: Could not find a version that satisfies the requirement ruamel (from versions: none)
ERROR: No matching distribution found for ruamel


In [1]:
# Import libraries
import cv2 # for reading images, draw bounding boxes
from ultralytics import YOLO
import gradio as gr

e:\Brad\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [2]:
# Define constants
BOX_COLORS = {
    "unchecked": (242, 48, 48),
    "checked": (38, 115, 101),
    "block": (242, 159, 5)
}
BOX_PADDING = 2

In [6]:
# Load models
DETECTION_MODEL = YOLO("../models/detector-model.pt")
CLASSIFICATION_MODEL = YOLO("../models/classifier-model.pt") # 0: block, 1: checked, 2: unchecked

def detect(image_path):
    """
    Output inference image with bounding box
    Args:
    - image: to check for checkboxes
    Return: image with bounding boxes drawn
    """
    image = cv2.imread(image_path)
    if image is None:
        return image

    # Predict on image
    results = DETECTION_MODEL.predict(source=image, conf=0.2, iou=0.8) # Predict on image
    boxes = results[0].boxes # Get bounding boxes

    if len(boxes) == 0:
        return image

    # Get bounding boxes
    for box in boxes:
        detection_class_conf = round(box.conf.item(), 2)
        detection_class = list(BOX_COLORS)[int(box.cls)]
        # Get start and end points of the current box
        start_box = (int(box.xyxy[0][0]), int(box.xyxy[0][1]))
        end_box = (int(box.xyxy[0][2]), int(box.xyxy[0][3]))
        box = image[start_box[1]:end_box[1], start_box[0]: end_box[0], :]

        # Determine the class of the box using classification model
        cls_results = CLASSIFICATION_MODEL.predict(source=box, conf=0.5)
        probs = cls_results[0].probs  # cls prob, (num_class, )
        classification_class = list(BOX_COLORS)[2 - int(probs.top1)]
        classification_class_conf = round(probs.top1conf.item(), 2)

        cls = classification_class if classification_class_conf > 0.9 else detection_class

        # 01. DRAW BOUNDING BOX OF OBJECT
        line_thickness = round(0.002 * (image.shape[0] + image.shape[1]) / 2) + 1
        image = cv2.rectangle(img=image,
                              pt1=start_box,
                              pt2=end_box,
                              color=BOX_COLORS[cls],
                              thickness = line_thickness) # Draw the box with predefined colors

        # 02. DRAW LABEL
        text = cls + " " + str(detection_class_conf)
        # Get text dimensions to draw wrapping box
        font_thickness =  max(line_thickness - 1, 1)
        (text_w, text_h), _ = cv2.getTextSize(text=text, fontFace=2, fontScale=line_thickness/3, thickness=font_thickness)
        # Draw wrapping box for text
        image = cv2.rectangle(img=image,
                              pt1=(start_box[0], start_box[1] - text_h - BOX_PADDING*2),
                              pt2=(start_box[0] + text_w + BOX_PADDING * 2, start_box[1]),
                              color=BOX_COLORS[cls],
                              thickness=-1)
        # Put class name on image
        start_text = (start_box[0] + BOX_PADDING, start_box[1] - BOX_PADDING)
        image = cv2.putText(img=image, text=text, org=start_text, fontFace=0, color=(255,255,255), fontScale=line_thickness/3, thickness=font_thickness)

    return image

iface = gr.Interface(fn=detect,
                     inputs=gr.Image(label="Upload scanned document", type="filepath"),
                     outputs="image")
iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.



0: 640x512 6 uncheckeds, 5 checkeds, 419.6ms
Speed: 6.0ms preprocess, 419.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 512)



Traceback (most recent call last):
  File "e:\Brad\.conda\Lib\site-packages\gradio\queueing.py", line 528, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Brad\.conda\Lib\site-packages\gradio\route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Brad\.conda\Lib\site-packages\gradio\blocks.py", line 1908, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Brad\.conda\Lib\site-packages\gradio\blocks.py", line 1485, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Brad\.conda\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Brad\.co

In [ ]:
image = cv2.imread('/content/Checkbox-Detection/demo/10_letter.jpg')

# Predict on image
results = DETECTION_MODEL.predict(source=image, conf=0.2, iou=0.8) # Predict on image
boxes = results[0].boxes # Get bounding boxes


0: 640x512 6 uncheckeds, 5 checkeds, 2707.6ms
Speed: 22.3ms preprocess, 2707.6ms inference, 36.2ms postprocess per image at shape (1, 3, 640, 512)


In [ ]:
results[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.])
conf: tensor([0.9373, 0.9366, 0.9280, 0.9081, 0.8952, 0.8761, 0.8433, 0.8328, 0.8186, 0.6927, 0.6780])
data: tensor([[1.1574e+00, 2.0375e+02, 3.0585e+01, 2.3313e+02, 9.3732e-01, 1.0000e+00],
        [2.8671e+02, 2.2480e+02, 3.1654e+02, 2.5408e+02, 9.3662e-01, 0.0000e+00],
        [6.3659e+02, 6.4262e+02, 6.6578e+02, 6.7177e+02, 9.2799e-01, 0.0000e+00],
        [5.5760e+02, 9.4344e+02, 5.9406e+02, 9.7640e+02, 9.0806e-01, 1.0000e+00],
        [5.0710e+02, 8.3794e+01, 5.4468e+02, 1.2496e+02, 8.9522e-01, 1.0000e+00],
        [1.3255e+02, 6.8566e+02, 1.6081e+02, 7.1471e+02, 8.7606e-01, 0.0000e+00],
        [7.2441e+02, 8.8723e+02, 7.5936e+02, 9.2356e+02, 8.4326e-01, 0.0000e+00],
        [6.3475e+02, 4.9842e+02, 6.6584e+02, 5.2814e+02, 8.3283e-01, 1.0000e+00],
        [2.2190e+01, 8.1635e+02, 4.8225e+01, 8.4416e+02, 8.1863e-01, 0.0000e+00],
        [6.9330e+02, 8.4680e+02, 7.10